In [2]:
from pyspark.sql.functions import col, sum, to_date, to_timestamp, regexp_replace, date_format 
from pyspark.sql.types import IntegerType, DoubleType, FloatType

In [3]:
dir_hdfs = "hdfs://namenode:9000/tmp/amd/locatel0311"

In [4]:
df = spark.read.csv(dir_hdfs)

In [5]:
df = spark.read.csv(dir_hdfs, header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- id_folio: string (nullable = true)
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tipo_de_entrada: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- fecha_concluido: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- 0311_colonia_registro: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [9]:
print(f"Numero de Renglones: {df.count()}")
print(f"Numero de Columnas: {len(df.columns)}")
print(f"Columnas {df.columns}")

Numero de Renglones: 2323089
Numero de Columnas: 15
Columnas ['id_folio', 'fecha_solicitud', 'hora_solicitud', 'tipo_de_entrada', 'tema_solicitud', 'sexo', 'edad', 'estatus', 'fecha_concluido', 'alcaldia', 'colonia_datos', '0311_colonia_registro', 'codigo_postal_solicitud', 'latitud', 'longitud']


In [12]:
df.show(truncate=False)

+--------+---------------+--------------+---------------------+---------------------------------------+---------------+----+--------+---------------+--------+-------------+---------------------+-----------------------+-------+--------+
|id_folio|fecha_solicitud|hora_solicitud|tipo_de_entrada      |tema_solicitud                         |sexo           |edad|estatus |fecha_concluido|alcaldia|colonia_datos|0311_colonia_registro|codigo_postal_solicitud|latitud|longitud|
+--------+---------------+--------------+---------------------+---------------------------------------+---------------+----+--------+---------------+--------+-------------+---------------------+-----------------------+-------+--------+
|- 244841|2020-01-11     |08:17:58      |VICTORIA             |FUGA DE AGUA                           |NO ESPECIFICADO|NA  |ATENDIDO|2020-07-03     |NA      |NA           |NA                   |NA                     |0      |0       |
|- 265596|2020-01-28     |12:10:42      |VICTORIA       

In [34]:
datos = df.select(['fecha_solicitud' , 'hora_solicitud' , 'tema_solicitud' , 'sexo' , 'edad' , 'estatus' ,
'alcaldia', 'colonia_datos' , 'codigo_postal_solicitud' , 'latitud' , 'longitud'])

In [35]:
datos.printSchema()

root
 |-- fecha_solicitud: string (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: string (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [36]:
datos.count()

2323089

In [37]:
datos.filter('sexo ="FEMENINO"').count()

829463

In [38]:
datos.select(['fecha_solicitud' , 'hora_solicitud' , 'tema_solicitud' , 'edad' , 'estatus' ,
'alcaldia', 'colonia_datos']).filter('sexo ="FEMENINO"').orderBy('fecha_solicitud').show()

+---------------+--------------+--------------------+----+--------+-----------------+--------------------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|edad| estatus|         alcaldia|       colonia_datos|
+---------------+--------------+--------------------+----+--------+-----------------+--------------------+
|     2019-06-02|      16:50:47|           ALUMBRADO|  29|ATENDIDO|       CUAUHTEMOC|      ROMA NORTE III|
|     2019-06-02|      23:30:08|       FALTA DE AGUA|  50|ATENDIDO|               NA|                  NA|
|     2019-06-02|      17:28:06|      MEDIO AMBIENTE|  19|ATENDIDO|               NA|                  NA|
|     2019-06-02|      14:40:34|        FUGA DE AGUA|  38|ATENDIDO|          TLALPAN|FLORESTA-PRADO-VE...|
|     2019-06-02|      17:33:25|MANTENIMIENTO VÍA...|  71|ATENDIDO|       IZTAPALAPA|          MODELO (U)|
|     2019-06-02|      15:17:44|              BACHEO|  66|ATENDIDO|         COYOACAN|  AJUSCO HUAYAMILPAS|
|     2019-06-02|      17:40:36|     

In [39]:
type(datos)

pyspark.sql.dataframe.DataFrame

In [40]:
#Transformar Columnas Cast
datos = datos.withColumn("fecha_solicitud", to_date(col("fecha_solicitud"), "yyy-MM-dd"))\
        .withColumn("hora_solicitud", date_format(col("hora_solicitud"), "HH:mm:ss"))\
        .withColumn("edad", col("edad").cast(FloatType()))\
        .withColumn("latitud", col("latitud").cast(DoubleType()))\
        .withColumn("longitud", col("longitud").cast(DoubleType()))\

datos.printSchema()
datos.show()

root
 |-- fecha_solicitud: date (nullable = true)
 |-- hora_solicitud: string (nullable = true)
 |-- tema_solicitud: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- edad: float (nullable = true)
 |-- estatus: string (nullable = true)
 |-- alcaldia: string (nullable = true)
 |-- colonia_datos: string (nullable = true)
 |-- codigo_postal_solicitud: string (nullable = true)
 |-- latitud: double (nullable = true)
 |-- longitud: double (nullable = true)

+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-----------------------+-------+--------+
|fecha_solicitud|hora_solicitud|      tema_solicitud|           sexo|edad| estatus|alcaldia|colonia_datos|codigo_postal_solicitud|latitud|longitud|
+---------------+--------------+--------------------+---------------+----+--------+--------+-------------+-----------------------+-------+--------+
|     2020-01-11|      08:17:58|        FUGA DE AGUA|NO ESPECIFICADO|NULL|ATENDIDO| 